# 通过NVIDIA DIGITS进行目标检测

## 概述

在本次实验中我们将通过多个实例说明如何使用DIGITS和 Caffe检测航拍图像中的目标。具体使用的是NOAA的露脊鲸识别竞赛(https://www.kaggle.com/c/noaa-right-whale-recognition) 的例子，参赛者被要求在海洋的航空影像中识别到鲸鱼。

图1显示了一个包含母露脊鲸和幼鲸的示例图像：

![Right whale example](images/right_whale_example.png)
<h4 align="center">图1 母露脊鲸和幼鲸</h4> 


我们将解决一个稍有不同的问题。不仅仅是要识别鲸鱼的存在，我们还要训练一个卷积神经网络（CNN）来定位鲸鱼在图像中的位置。不论鲸鱼在不在图像里，这类定位问题时常被称为目标检测。在最初的竞赛中，许多成功的参赛选手发现，在试图用裁剪和标准化的图像来识别鲸鱼之前，首先在图像中发现并定位鲸鱼，可以提高他们的得分。

## 目标检测方法1：滑动窗口

有多种方法可以用卷积神经网络（CNN）来检测和定位图像中的物体。最简单的方法是首先在可以区分目标和非目标实体的图像块上训练CNN分类器。图2显示了一个CNN的架构，其能够从背景图块中区分出鲸鱼的斑块。

![patch based model](images/patch_based.png)
<h4 align="center">图2：基于图像块进行训练的CNN分类器</h4> 

通过使用这种类型的分类器，我们可以在较大的图像中检查每个图像块，还可以使用重叠的图像块，以确定是否有鲸鱼存在。让我们按照这个方法往下做。

在这个实验中我们有两个数据集可用。第一种是包含鲸鱼的广阔海域的图集。这个图集位于data_336x224。第二个数据集是大约4500个鲸鱼面部的局部图，另外4500个来自同一个图像的随机局部图。这个数据集在data/train/faces 和 data/train/background目录下。我们将在DIGITS中使用第二个数据集训练我们的分类器。

<a id='question1'></a>
### 问题 1

从背景数据集中使用随机局部图可能会有什么问题？

答案: [请点击这里](#answer1)

[点击这里](/digits/) 打开DIGITS.

首先，我们需要将数据集导入到DIGITS中。使用 Datasets->Images下拉菜单，选择 “classification” 。第一次提示您输入用户名。输入一个您喜欢的标准用户名。当“New Image Classification Dataset”面板打开，使用下面的预处理选项:

![whale_faces_import](images/whale_faces_import.png)

程序将导入这个256x256 像素的face/not-face数据集并分出25%作为验证数据集-DIGITS会自动从图像文件夹结构中区分这两个类别的名字。


DIGITS需要几分钟来完成数据导入，一旦导入完成，如果返回到DIGITS主屏幕，然后重新进入whale_faces数据集，点击 "Explore the db" 按钮查看每个类中的图像示例。您将看到如下所示:

![whale face examples](images/whale_face_examples.png)

现在我们将在这个数据集上训练一个简单的两级分类CNN分类器。返回到DIGITS主屏幕并使用Models->Images下拉菜单并选择 “classification” 模型。在打开的 “New Image Classification Model” 面板上，我们将保留大多数选项的默认值。您只需要自定义以下内容:

* 选择刚刚创建的whale_faces数据集 
* 选择标准的网络“AlexNet”
* 将训练时长设为5个epoch
* 给模型起一个名字，取名叫“whale_faces_baseline”

面板如下图:

![DIGITS New Image Classification Model panel](images/whale_faces_digits_model.png)

点击 "Create" 按钮开始训练模型。

您应该看到一个实时更新图表，显示模型训练损失值和验证集的损失值以及准确度。随着训练的进行损失值将减小，准确度将提高。训练需要几分钟时间才能完成。最后，您应该看到验证的准确率在98%左右-我们得到了一个非常好的whale face/non-face模型。

您可以把URL `/dli/data/whale/data/train/face/w_2606.jpg` 以文本的方式输入， 并点击"Classify One"按钮，以此来测试针对单个图像的分类。请务必检查"Show visualizations and statistics"框，查看CNN是如何对图像响应来进行分类的。

现在我们有了这个模型，我们将在这个实验中使用它，在广阔的航拍图像上用一个滑动窗口来检测鲸鱼面部。您刚刚用DIGITS训练出的模型的作业编号，可以在作业页面的URL尾部找到。例如如果URL是 `localhost/models/20160525-014512-ce40` 那么作业编号就是 `20160525-014512-ce40`.  您需要用到这个编号 - 在后面的代码中我们将用参考模型的作业编号设置MODEL_JOB_NUM，用数据集的作业编号设置DATASET_JOB_NUM。

修改下面的代码为您的模型来设置正确的MODEL_JOB_NUM，并为您的数据集来设置正确的DATASET_JOB_NUM，然后执行这个程序。程序大概需要30秒时间来执行，输出结果将显示从测试图片中随机选择的一个区域以及一个预测结果数组，这个数组包含了针对每一个256*256大小的非重叠栅格的预测结果。

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import caffe
import time

MODEL_JOB_NUM = '20180918-192235-f654'  ## Remember to set this to be the job number for your model
DATASET_JOB_NUM = '20180918-192003-efbb'  ## Remember to set this to be the job number for your dataset

MODEL_FILE = '/dli/data/digits/' + MODEL_JOB_NUM + '/deploy.prototxt'                 # Do not change
PRETRAINED = '/dli/data/digits/' + MODEL_JOB_NUM + '/snapshot_iter_270.caffemodel'    # Do not change
MEAN_IMAGE = '/dli/data/digits/' + DATASET_JOB_NUM + '/mean.jpg'                      # Do not change

# load the mean image
mean_image = caffe.io.load_image(MEAN_IMAGE)

# Choose a random image to test against
RANDOM_IMAGE = str(np.random.randint(10))
IMAGE_FILE = '/dli/data/whale/data/samples/w_' + RANDOM_IMAGE + '.jpg'                   

# Tell Caffe to use the GPU
caffe.set_mode_gpu()
# Initialize the Caffe model using the model trained in DIGITS
net = caffe.Classifier(MODEL_FILE, PRETRAINED,
                       channel_swap=(2,1,0),
                       raw_scale=255,
                       image_dims=(256, 256))

# Load the input image into a numpy array and display it
input_image = caffe.io.load_image(IMAGE_FILE)
plt.imshow(input_image)
plt.show()

# Calculate how many 256x256 grid squares are in the image
rows = input_image.shape[0]/256
cols = input_image.shape[1]/256

# Initialize an empty array for the detections
detections = np.zeros((rows,cols))

# Iterate over each grid square using the model to make a class prediction
start = time.time()
for i in range(0,rows):
    for j in range(0,cols):
        grid_square = input_image[i*256:(i+1)*256,j*256:(j+1)*256]
        # subtract the mean image
        grid_square -= mean_image
        # make prediction
        prediction = net.predict([grid_square])
        detections[i,j] = prediction[0].argmax()
end = time.time()
        
# Display the predicted class for each grid square
plt.imshow(detections)

# Display total time to perform inference
print 'Total inference time: ' + str(end-start) + ' seconds'


当您多次运行上面的代码时，您会发现，在某些情况下，使用这个基准模型和滑动窗口法能够找到鲸鱼的面部——多数情况下，它会发现大量的鲸鱼。但您也会看到此模型很容易被海浪或从海面反射出来的阳光所迷惑。

<a id='question2'></a>
### 问题2

有哪些方法可以提高该模型的分类精度？

答案: [请点击这里](#answer2)

事实上，我们使用一个滑动窗口与非重叠的栅格意味着它很可能是我们的一些栅格只能部分包含一个鲸鱼的面部，这会导致错误分类。不幸的是，如果我们增加了重叠的栅格，这个滑动窗口方法的运算时间将急剧增加。我们还需要一个解决方案将重叠分类结合到最终分类“热图”中去-一种流行的方法是非最大抑制（NMS）算法。

<a id='question3'></a>
### 问题 3

我们怎样才能抵消掉重叠栅格所需增加的计算时间呢？

答案: [请点击这里](#answer3)

### 具有挑战性的可选练习：

<a id='question-optional-exercise'></a>

1. 栅格大小为256*256，修改代码，增加栅格之间的重叠并获得更细的分类图。

2. 将代码修改为批处理多个栅格，以便传入网络进行预测。

答案: [请点击这里](#answer-optional-exercise)

正如我们所看到的这种滑动窗口方法的优点是，我们可以训练一个检测器，只使用基于局部块来训练数据（这种方法具有更广泛的使用）。但也有几个缺点:

* 预测速度慢，特别是栅格之间存在较大的重叠，导致大量冗余的计算
* 对于产生一个平衡的训练数据集是一个挑战，非常容易造成误报造成混乱
* 难以达到目标检测的尺度不变性

## 目标检测方法2：候选生成和分类

我们不会实际演示这第二种方法，但是为了完整性，我们在这里描述它。我们不采用以滑动窗口的方式实现CNN分类，您可以使用一些更便宜、更灵敏的样式，但容易产生候选检测误报的算法。用于这个流程的算法的例子是级联分类器和选择性搜索。将这些候选检测体传递给CNN按目标类型进行分类或者作为背景噪声过滤掉。

这些候选生成算法，通常会生成比在滑动窗口方法中测试所需要的栅格要少得多的图像块来通过CNN分类。此外，这些候选的检测可以进行批处理后输入CNN，以便从并行运算中得到性能提升。

图3显示了如何在车辆检测场景中使用此方法。

![candidate generation example](images/candidate_generation_example.png)
<h4 align="center">图3：带有候选生成预处理的CNN分类器</h4> 

这种方法的优点是：

* 候选检测体的数目变少，使得测试加速
* 根据候选生成算法，我们可以获得更精确的目标定位

这种方法的缺点是：

* 多级处理通道变得更加复杂
* 需要候选生成构建或训练一个附加模型
* 较高的误报率
* 基于候选生成的数量，推理时间是一个变量

## 目标检测方法3：全卷积网络（FCN）

正如前面提到的，在重叠窗口的滑动窗口法中存在大量冗余计算。幸好有一个巧妙的方法能避免这种冗余。对于像Alexnet这样的卷积分类的末级通常采用全连接层，可以很简单的用卷积层替换。这些替换层具有与前一层的特征映射输出相同的卷积滤波器，滤波器的数目等于它替换的全连接层中的神经元的数目。这种替换的好处是可以将不同大小的图像输入到网络中进行分类。如果输入图像小于网络的期望图像大小（称为网络的感受野），那么我们仍然能够获得这副图像的单一分类。然而，如果图像比感受野大，我们将得到一个热图的分类，非常像我们从滑动窗口获取分类的方法。

让我们来看看Alexnet的`fc6` 层是如何工作的。
您可以用DIGITS来检查到 `fc6`输入形状: 使用您在这个实验第一部分训练得到的CNN分类器, 勾选 "Show visualizations and statistics" 来测试任意图像。
您将看到如下图4所示:

![alexnet layers](images/alexnet_layers.png)
<h4 align="center">图4：Alexnet FC6层</h4>

可以看到`fc6`从`pool5`接收输入.
在`pool5` 的激活形状是`256*6*6`.

在`fc6`的激活的形状是4096，这意味着`fc6`有4096个输出神经元。把`fc6`转化为一个等效的卷积层，我们将创建一个6×6卷积核的卷积层和4096输出的特征图。

让我们用鲸鱼检测问题来解决验证。返回到DIGITS，并使用模型屏幕右上角的 “Clone” 按钮克隆基准鲸鱼面部检测模型。

在Alexnet网络结构边选择“Customize”。

<a id='exercise1'></a>
### 练习:

采用具有256 6x6卷积过滤器和零填充的卷积层代替fc6全连接层(InnerProduct)。同样，用卷积层替代fc7和fc8层。看看能否成功地训练这个模型。

答案: [请点击这里](#answer4)

一旦做出了所需的更改，模型就进行训练，并达到大约98%的验证精度。

我们可以使用该模型对一整幅大型航拍图像直接计算分类热图。我们仍然可以有效的使用一个滑动窗口分类，但所有的滑动窗口都是在FCN构架下进行高效的处理。

运行下面的代码来查看这个操作。同样，您需要获取模型的作业编号并在代码中替换。

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import caffe
import copy
from scipy.misc import imresize
import time

JOB_NUM = '20180918-192235-f654'  ## Remember to set this to be the job number for your model

MODEL_FILE = '/dli/data/digits/' + JOB_NUM + '/deploy.prototxt'                 # Do not change
PRETRAINED = '/dli/data/digits/' + JOB_NUM + '/snapshot_iter_270.caffemodel'    # Do not change
    
# Choose a random image to test against
RANDOM_IMAGE = str(np.random.randint(10))
IMAGE_FILE = '/dli/data/whale/data/samples/w_' + RANDOM_IMAGE + '.jpg'                  

# Tell Caffe to use the GPU
caffe.set_mode_gpu()

# Load the input image into a numpy array and display it
input_image = caffe.io.load_image(IMAGE_FILE)
plt.imshow(input_image)
plt.show()

# Initialize the Caffe model using the model trained in DIGITS
# This time the model input size is reshaped based on the randomly selected input image
net = caffe.Net(MODEL_FILE,PRETRAINED,caffe.TEST)
net.blobs['data'].reshape(1, 3, input_image.shape[0], input_image.shape[1])
net.reshape()
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2,0,1))
transformer.set_channel_swap('data', (2,1,0))
transformer.set_raw_scale('data', 255.0)

# This is just a colormap for displaying the results
my_cmap = copy.copy(plt.cm.get_cmap('jet')) # get a copy of the jet color map
my_cmap.set_bad(alpha=0) # set how the colormap handles 'bad' values

# Feed the whole input image into the model for classification
start = time.time()
out = net.forward(data=np.asarray([transformer.preprocess('data', input_image)]))
end = time.time()

# Create an overlay visualization of the classification result
im = transformer.deprocess('data', net.blobs['data'].data[0])
classifications = out['softmax'][0]
classifications = imresize(classifications.argmax(axis=0),input_image.shape,interp='bilinear').astype('float')
classifications[classifications==0] = np.nan
plt.imshow(im)
plt.imshow(classifications,alpha=.5,cmap=my_cmap)
plt.show()

# Display total time to perform inference
print 'Total inference time: ' + str(end-start) + ' seconds'

### 重要：在开始后面实验之前执行下面单元的程序

In [ ]:
try:
    del net
    del transformer
    del classifications
    del input_image
except:
    print "Memory already freed"

当运行上面的代码多次，您会看到，在许多情况下，FCN定位鲸鱼的面部具有比滑动窗口法具有更高的精度。它经常可以发现较大数量的鲸鱼，不过有时仍然会被海浪或从海面反射出来的阳光所迷惑。同样，通过使用适当的数据增强，可以减少由背景杂波和鲸鱼身体引起的误报。

注意，FCN总推理时间约为1.5秒，而滑动窗口的方法花了10秒，因此，我们在少量的时间内获得了更好质量的结果！如果我们希望在诸如飞机上将探测器部署在实时应用程序中，这将非常有优势，因为我们得到的一个单一模型可以一个在高效通道中完成检测和分类。

除了通过数据增强来应对我们的训练数据，还有几个常用的提高FCN分类准确度和定位精度的途径。最常见的是在不同的尺度上多次通过网络传递输入图像。这提高了模型对目标外观尺度变化的耐受性。通过使用多个版本的输入图像和分类网络同时进行分类的热图的合并过程，可以大大提高最终的分类和检测结果。这个方法所为人熟知的例子，是本文提出的OverFeat（图5）。

![overfeat](images/overfeat.png)
<h4 align="center">图5：为提高检测的分辨率OverFeat方法</h4> 

### 练习:  

现在返回并修改FCN测试代码查看到通过不同大小的图像输入到网络的效果。首先，选择一个固定的输入图像来测试。然后手动调整该图像大小，然后将其传递到网络中。这样做几次，并比较其对检测准确度的效果。

## 目标检测方法4：DetectNet

最后一类目标检测方法是训练一个CNN网络，以便同时对在一副图像中每个位置可能出现的目标进行分类。并通过回归法预测该目标的相应边界框。例如:

![yolo](images/yolo.png)

这种方法有很大的优点：
* 只有一次的简单检测、分类和边界框回归反馈通道
* 非常低的延迟
* 由于强而大量的背景训练数据，误报率非常低

为了训练这种类型的网络，需要专门的训练数据，所有感兴趣的目标都用精确的边界框标记。这种类型的训练数据比较稀少，成本也很高；然而，如果这种类型的数据可以用于目标检测问题，这几乎是最好的方法。图6显示了用于车辆检测场景的标记训练样本的示例。
![kespry example](images/kespry_example.png)
<h4 align="center">图6：三个类别的目标检测场景的标签数据</h4> 

最近发布的DIGITS 4增加了这类模型训练的功能，并提供了一个新的 “standard network” -称为 DetectNet - 作为例子。我们将使用 DetectNet在全尺寸海洋航拍图像中训练露脊鲸检测器。 

在训练单个CNN作为目标检测和边界框回归中主要面临的挑战是处理不同图像中存在不同数量的目标这一情形，在某些情况下，图像中可能一个目标对象都没有。 DetectNet通过将一副具有一系列边界框的注释的图像转换成一个固定维度的描述。我们直接通过CNN尝试预测。图6显示了在一个单独的分类目标检测问题中，数据是如何映射到这个描述的。

![detectnet data rep](images/detectnet_data.png)
<h4 align="center">图7：DetectNet数据描述</h4> 

DetectNet实际上是上面所述的FCN，但是需要配置以产生精确的数据描述作为它的输出。DetectNet大部分层是著名的GoogLeNet网络。

![detectnet training architecture](images/detectnet_training.png)
<h4 align="center">图8: DetectNet训练构架</h4> 

为了进行这个实验，我们已经准备了全尺寸的海洋航空图像，以相应的目录和格式训练DetectNet，我们首先需要把数据导入DIGITS。使用Datasets->Images下拉菜单，选择"object detection" 数据集。当"New Object Detection Dataset" 面板打开，使用下列预处理选项。

![OD data ingest](images/OD_ingest.png)

现在我们先来看看如何在此数据集训练DetectNet。在这个数据集上进行DetectNet全部训练需要几个小时，为此我们提供一个已经训练好了的模型来做实验。返回到DIGITS主屏幕并使用Models模型选项卡。打开 “whale_detectnet” 模型并克隆。进行以下更改：

* 选择新创建的“whales_detectnet”数据集
* 将训练epoch数目改为3 
* 将批处理大小更改为10

点击"Visualize"按钮，查看网络构架  

当准备训练时，给模型起一个新的名字如“whale_detectnet_2”并单击“create”。仅为3个epochs的训练，此模型仍需要大约8分钟，但您应该看到卷积和边界框的训练和验证损失值已经减少了。您也会看到平均精度（mAP）得分开始上升。mAP是一个衡量网络探测鲸鱼面部的能力以及为验证数据集评估边界框的准确程度的综合测量。

一旦模型完成了训练返回到预先训练 ”whale_detectnet” 模型。可以看到，在100个训练epochs之后，该模型不仅收敛到低的训练和验证损失值。而且有很高的mAP分数。让我们测试这个经过训练的模型，验证它是否能找到鲸鱼的面部。

只需将可视化方法设置为“Bounding boxes”，然后粘贴以下图像路径:  `/dli/data/whale/data_336x224/val/images/000000118.png`.  确认选择了 "Show visualizations and statistics"复选框，并且点击"Test One"。  您将看到DetectNet成功检测到鲸鱼的面部并绘制一个边界框，如图所示:

![detectnet success](images/detectnet_success.png)

从 `/dli/data/whale/data_336x224/val/images/` 目录下任意测试其他图像。  可以看到DetectNet通过紧紧环绕的边界框能够准确检测大多数鲸鱼面部并具有非常低的误报率。此外，DetectNet推理非常快。执行下面用Caffe命令行接口来执行采用DetectNet构架的推理标准检测程序，您将会看到通过DetecNet对336*224大小的图片处理一次只需要22ms。

In [ ]:
!caffe time --model /dli/data/whale/deploy.prototxt --gpu 0

## 问题答案:

<a id='answer1'></a>
### 答案1

随机图像块可能最终也包含着鲸鱼的面部。这是不可能的，因为鲸鱼面部通常只是图像的一小部分，我们拥有的大量的随机背景图块，几乎完全不包含鲸鱼脸。我们也可以在我们的背景中找到鲸鱼的身体和尾巴，但这是我们对鲸鱼面部定位感兴趣的地方。

[点击这里](#question1) 返回问题 1

<a id='answer2'></a>
### 答案2

尝试将使用大量的随机选择的非面部的图块，并通过随机旋转，翻转和缩放来增强现有的面部图块，以达到数据集均衡的效果。还可以用大量的带有训练参数的模型，如GoogleNet。

[点击这里](#question2) 返回问题 2

<a id='answer3'></a>
### 答案3

我们可以一次批出多个栅格，作为批处理送入网络，这样我们就可以进一步利用并行，并从GPU获得计算加速。

[点击这里](#question3) 返回问题3

<a id='answer-optional-exercise'></a>

### 自选练习答案


In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import caffe
import time

MODEL_JOB_NUM = '20160920-092148-8c17'  ## Remember to set this to be the job number for your model
DATASET_JOB_NUM = '20160920-090913-a43d'  ## Remember to set this to be the job number for your dataset

MODEL_FILE = '/dli/data/digits/' + MODEL_JOB_NUM + '/deploy.prototxt'                 # Do not change
PRETRAINED = '/dli/data/digits/' + MODEL_JOB_NUM + '/snapshot_iter_270.caffemodel'    # Do not change
MEAN_IMAGE = '/dli/data/digits/' + DATASET_JOB_NUM + '/mean.jpg'                      # Do not change

# load the mean image
mean_image = caffe.io.load_image(MEAN_IMAGE)

# Choose a random image to test against
RANDOM_IMAGE = str(np.random.randint(10))
IMAGE_FILE = '/dli/data/whale/data/samples/w_' + RANDOM_IMAGE + '.jpg' 

# Tell Caffe to use the GPU
caffe.set_mode_gpu()
# Initialize the Caffe model using the model trained in DIGITS
net = caffe.Classifier(MODEL_FILE, PRETRAINED,
                       channel_swap=(2,1,0),
                       raw_scale=255,
                       image_dims=(256, 256))

# Load the input image into a numpy array and display it
input_image = caffe.io.load_image(IMAGE_FILE)
plt.imshow(input_image)
plt.show()

# Calculate how many 256x256 grid squares are in the image
rows = input_image.shape[0]/256
cols = input_image.shape[1]/256

# Subtract the mean image
for i in range(0,rows):
    for j in range(0,cols):
        input_image[i*256:(i+1)*256,j*256:(j+1)*256] -= mean_image
        
# Initialize an empty array for the detections
detections = np.zeros((rows,cols))
        
# Iterate over each grid square using the model to make a class prediction
start = time.time()
for i in range(0,rows):
    for j in range(0,cols):
        grid_square = input_image[i*256:(i+1)*256,j*256:(j+1)*256]
        # make prediction
        prediction = net.predict([grid_square])
        detections[i,j] = prediction[0].argmax()
end = time.time()
        
# Display the predicted class for each grid square
plt.imshow(detections)
plt.show()

# Display total time to perform inference
print 'Total inference time (sliding window without overlap): ' + str(end-start) + ' seconds'

# define the amount of overlap between grid cells
OVERLAP = 0.25
grid_rows = int((rows-1)/(1-OVERLAP))+1
grid_cols = int((cols-1)/(1-OVERLAP))+1

print "Image has %d*%d blocks of 256 pixels" % (rows, cols)
print "With overlap=%f grid_size=%d*%d" % (OVERLAP, grid_rows, grid_cols)

# Initialize an empty array for the detections
detections = np.zeros((grid_rows,grid_cols))

# Iterate over each grid square using the model to make a class prediction
start = time.time()
for i in range(0,grid_rows):
    for j in range(0,grid_cols):
        start_col = j*256*(1-OVERLAP)
        start_row = i*256*(1-OVERLAP)
        grid_square = input_image[start_row:start_row+256, start_col:start_col+256]
        # make prediction
        prediction = net.predict([grid_square])
        detections[i,j] = prediction[0].argmax()
end = time.time()
        
# Display the predicted class for each grid square
plt.imshow(detections)
plt.show()

# Display total time to perform inference
print ('Total inference time (sliding window with %f%% overlap: ' % (OVERLAP*100)) + str(end-start) + ' seconds'

# now with batched inference (one column at a time)
# we are not using a caffe.Classifier here so we need to do the pre-processing
# manually. The model was trained on random crops (256*256->227*227) so we
# need to do the cropping below. Similarly, we need to convert images
# from Numpy's Height*Width*Channel (HWC) format to Channel*Height*Width (CHW) 
# Lastly, we need to swap channels from RGB to BGR
net = caffe.Net(MODEL_FILE, PRETRAINED, caffe.TEST)
start = time.time()
net.blobs['data'].reshape(*[grid_cols, 3, 227, 227])

# Initialize an empty array for the detections
detections = np.zeros((rows,cols))

for i in range(0,rows):
    for j in range(0,cols):
        grid_square = input_image[i*256:(i+1)*256,j*256:(j+1)*256]
        # add to batch
        grid_square = grid_square[14:241,14:241] # 227*227 center crop        
        image = np.copy(grid_square.transpose(2,0,1)) # transpose from HWC to CHW
        image = image * 255 # rescale
        image = image[(2,1,0), :, :] # swap channels
        net.blobs['data'].data[j] = image
    # make prediction
    output = net.forward()[net.outputs[-1]]
    for j in range(0,cols):
        detections[i,j] = output[j].argmax()
end = time.time()
        
# Display the predicted class for each grid square
plt.imshow(detections)
plt.show()

# Display total time to perform inference
print 'Total inference time (batched inference): ' + str(end-start) + ' seconds'

[点击这里](#question-optional-exercise) 返回问题

<a id='answer4'></a>
### 答案4

通过下列用fc8取代fc6。设置将底部的blob类（Caffe基本类）的损失值，准确性和softmax层给`conv8`。

```layer {
  name: "conv6"
  type: "Convolution"
  bottom: "pool5"
  top: "conv6"
  param {
    lr_mult: 1.0
    decay_mult: 1.0
  }
  param {
    lr_mult: 2.0
    decay_mult: 0.0
  }
  convolution_param {
    num_output: 4096
    pad: 0
    kernel_size: 6
    weight_filler {
      type: "gaussian"
      std: 0.01
    }
    bias_filler {
      type: "constant"
      value: 0.1
    }
  }
}
layer {
  name: "relu6"
  type: "ReLU"
  bottom: "conv6"
  top: "conv6"
}
layer {
  name: "drop6"
  type: "Dropout"
  bottom: "conv6"
  top: "conv6"
  dropout_param {
    dropout_ratio: 0.5
  }
}
layer {
  name: "conv7"
  type: "Convolution"
  bottom: "conv6"
  top: "conv7"
  param {
    lr_mult: 1.0
    decay_mult: 1.0
  }
  param {
    lr_mult: 2.0
    decay_mult: 0.0
  }
  convolution_param {
    num_output: 4096
    kernel_size: 1
    weight_filler {
      type: "gaussian"
      std: 0.01
    }
    bias_filler {
      type: "constant"
      value: 0.1
    }
  }
}
layer {
  name: "relu7"
  type: "ReLU"
  bottom: "conv7"
  top: "conv7"
}
layer {
  name: "drop7"
  type: "Dropout"
  bottom: "conv7"
  top: "conv7"
  dropout_param {
    dropout_ratio: 0.5
  }
}
layer {
  name: "conv8"
  type: "Convolution"
  bottom: "conv7"
  top: "conv8"
  param {
    lr_mult: 1.0
    decay_mult: 1.0
  }
  param {
    lr_mult: 2.0
    decay_mult: 0.0
  }
  convolution_param {
    num_output: 2
    kernel_size: 1
    weight_filler {
      type: "gaussian"
      std: 0.01
    }
    bias_filler {
      type: "constant"
      value: 0.1
    }
  }
}
```

[点击这里](#exercise1) 返回练习

### Thanks and data attribution

在此感谢NOAA允许重复使用露脊鲸的图像用于本实验。

图像在下列许可编号下收集到:

MMPA 775-1600,      2005-2007

MMPA 775-1875,      2008-2013 

MMPA 17355,         2013-2018

照片来源: NOAA/NEFSC